In [221]:
from officelib.xllib import *
from officelib.const import xlconst as xlc
from pywintypes import com_error
from scripts.tools.issuetracker import IssuetrackerAPI

In [232]:
xl = Excel(new=False)
wb2 = xl.RecentFiles(1).Open()
wb1 = xl.RecentFiles(3).Open()
print(wb1.Name, wb2.Name)

IS00038B1 Rev A 3.0 software project progress.xlsm Traceability Matrix (Reqs and VI) with full FRS check 161109.xlsx


In [12]:
ws = wb2.ActiveSheet
cells = ws.Cells
cr = cells.Range
id_start = cr("A2")
id_end = id_start.End(xlc.xlDown)
frs_start = cr("P2")
frs_end = frs_start.Offset(id_end.Row - id_start.Row + 1, 1)
print(cr(id_start, id_end).Address, cr(frs_start, frs_end).Address)


$A$2:$A$1229 $P$2:$P$1229


In [222]:
api = IssuetrackerAPI('issue.pbsbiotech.com', 'nstarkweather', 'kookychemist')
issues = api.download_issues("pbssoftware")

In [223]:
relevant = []
for v in issues.values():
    if v.sprint_milestone == "3.0" and \
        v.tracker == "Specification" and \
        v.status not in ("Closed", "Rejected"):
            relevant.append(v) 


In [224]:
test_map = []
test_map2 = {}
test_data = cr(id_start, frs_end).Value
for row in test_data:
    id_test = row[0]
    iid_test = int(id_test)
    if iid_test == id_test:
        id_test = iid_test
    list_web = row[-1] or ""
    frs = list_web.split("\n")
    
    for f in filter(None, frs):
        f=f.strip()
        test_map.append((f, id_test))
        flist = test_map2.get(f, None)
        if flist is None:
            flist = test_map2[f] = []
        flist.append(id_test)

com_error: (-2147352567, 'Exception occurred.', (0, None, None, None, 0, -2146827284), None)

In [ ]:
def frs_nums(s):
    pos = s.find("FRS")
    num = s[pos+3:]
    return num.split(".")

def num_tuple(s):
    return tuple(int(i) for i in frs_nums(s))
    
def sort_key(t):
    return num_tuple(t[0])

        
test_map.sort(key=sort_key)
test_map

In [ ]:
import re
toplevel_match = re.compile(r"^\>?[\+\*]{2}(FRS\d+)[\+\*]{2}.*$").match
subitem_match = re.compile(r"^\>?[\+\*]+\s\*(FRS[\d\.]+)\:?\*\:?.*$").match
all_frs = []
for v in relevant:
    lines = v.description.splitlines()
    for line in lines:
        for match in (toplevel_match, subitem_match):
            m = match(line)
            if m:
                frs = m.group(1)
                all_frs.append(frs)
all_frs.sort(key=sort_frs_item)
all_frs

In [225]:
def dump(node, level=0):
    for k, v in sorted(node.children.items()):
        print(" "*level + str(v.id))
        dump(v, level+1)

class Node():
    def __init__(self, id, parent):
        self.id = id
        self.parent = parent
        self.children = {}
        self.tests = []
        
    def get(self, id):
        child = self.children.get(id, None)
        if child is None:
            child = self.mk_child(id)
            self.children[id] = child
        return child
    
    def mk_child(self, id):
        return Node("%s.%s"%(self.id, id), self)
    
    def _find(self, path):
        id = path.pop()
        child = self.get(id)
        if not path:
            return child
        return child._find
    
    def iter(self):
        for k, v in sorted(self.children.items()):
            yield v
            yield from v.iter()
        
    
frs_match = re.compile(r"(.*FRS\d+)\.?([\d\.]*)").match
    
class Root(Node):
    def __init__(self):
        super().__init__("", None)
        
    def mk_child(self, id):
        return Node(id, self)
        
    def add(self, key):
        frs, nums = frs_match(key).groups()
        child = self.get(frs)
        if not nums:
            return child
        path = [int(i) for i in nums.split(".")]
        path.reverse()
        node = child
        while path:
            id = path.pop()
            node = node.get(id)
        return node
        
root = Root()
for f in all_frs:
    id_test = test_map2.get(f, ())
    node = root.add(f)
    
    
#dump(root)
ids = [v.id for v in root.iter()]
print(set(ids) - set(all_frs))
print(set(all_frs) - set(ids))

{'FRS2877.1.1', 'FRS2877.2.1', 'FRS2856.2.8', 'FRS2851.5'}
set()


In [102]:
def num_tuples(f1, f2):
    return num_tuple(f1), num_tuple(f2)

def is_parent(f1, f2):
    t1, t2 = num_tuples(f1, f2)
    for a, b in zip(t1, t2):
        if a != b:
            return False
    return len(t1) < len(t2)

def is_child(f1, f2):
    t1, t2 = num_tuples(f1, f2)
    for a, b in zip(t1, t2):
        if a != b:
            return False
    return len(t2) > len(t1)

paste_data = []
for s in all_frs:
    tests = test_map2.get(s, ())
    tests = sorted(tests)
    test_str = "\n".join(map(str,tests))
    paste_data.append((s, test_str, "'True" if test_str else ""))

In [112]:
try:
    ws2 = wb2.Worksheets("Matrix")
    ws2.UsedRange.Clear()
except com_error:
    ws2 = wb2.Worksheets.Add()
    ws2.Name = "Matrix"

cells2 = ws2.Cells
cr2 = cells2.Range
    
headers = [("FRS Number", "id_test", "Tested")]
cr2(cr2("A1"), cr2("A1").Offset(1, len(headers[0]))).Value = headers

frs_start = cr2("A2")
frs_end = frs_start.Offset(len(paste_data), 1)

id_start = frs_start.Offset(1, 2)
id_end = frs_end.Offset(1, 2)

tested_start = id_start.Offset(1,2)
tested_end = id_end.Offset(1,2)

paste_start = frs_start
paste_end = tested_end
paste_start.Address, paste_end.Address

('$A$2', '$C$609')

In [113]:
paste_range = cr2(paste_start, paste_end)
paste_range.Value = paste_data

cr2(frs_start, frs_end).VerticalAlignment = xlc.xlTop
cr2(id_start, id_end).HorizontalAlignment = xlc.xlRight
cr2(tested_start, tested_end).HorizontalAlignment = xlc.xlRight
cr2(tested_start, tested_end).VerticalAlignment = xlc.xlTop

'$A$2:$B$20'